<div style="text-align:center; margin-top: 1cm;">
    <h2>Київський політехнічний інститут імені Ігоря Сікорського</h2>
    <h2>Приладобудівний факультет</h2>
    <h3>Кафедра автоматизації та систем неруйнівного контролю</h3>
    <br><br><br>
</div>

<div style="text-align:center; margin-top: 5cm;">
    <h2>Лабораторна робота №4 </h2>
    <h2>з предмету "Комп'ютерний зір"</h2>
</div>

<div style="text-align:right; margin-top: 5cm;">
<p>Студент: Погорєлов Богдан<br>
    Група: ПК-51мп<br>
</p>
</div>
<div style="text-align:center; margin-top: 5cm;">
2025 рік  <br><br><br><br>
</div>

Розробити програму, яка виконує такі дії з цифровим ЦВ:
1. Програма повинна виконувати супроводження певного об’єкту за допомогою кореляційних методів у
вхідному ЦВ. У вікні відображення ЦВ програма виводить інформацію про координати об’єкту та
відображає його положення. Також програма повинна працювати у таких режимах:
- супроводження об’єкту у монохромному вхідному ЦВ;
- супроводження об’єкту при низько частотній фільтрації монохромного вхідного ЦВ;
- супроводження об’єкту при високо частотній фільтрації монохромного вхідного ЦВ;
2. Програма повинна записувати вказані користувачем кадри вихідного ЦВ у файли ЦЗ. Ім’я файлу ЦЗ
повинно містити порядковий номер кадру у вихідному ЦВ.

In [ ]:
import cv2
import numpy as np
import os

# --- НАЛАШТУВАННЯ ---
# Папка для збереження кадрів
OUTPUT_FOLDER = "saved_frames"
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

# Глобальні змінні
drawing = False      # Чи малюємо ми зараз прямокутник
roi_selected = False # Чи обрано об'єкт для стеження
ix, iy = -1, -1      # Початкові координати миші
bbox = (0, 0, 0, 0)  # Координати виділення (x, y, w, h)
template = None      # Шаблон (картинка) об'єкта
frame_count = 0      # Лічильник кадрів

# Режими відображення
MODE_MONO = 0
MODE_LOW_PASS = 1
MODE_HIGH_PASS = 2
current_mode = MODE_MONO
mode_names = {0: "Monochrome", 1: "Low Pass (Blur)", 2: "High Pass (Edge)"}

def mouse_callback(event, x, y, flags, param):
    """Обробка натискань миші для виділення об'єкта."""
    global ix, iy, drawing, bbox, roi_selected, template, frame_gray

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        roi_selected = False
        ix, iy = x, y

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            # Оновлюємо прямокутник під час руху
            bbox = (min(ix, x), min(iy, y), abs(x - ix), abs(y - iy))

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        bbox = (min(ix, x), min(iy, y), abs(x - ix), abs(y - iy))
        w, h = bbox[2], bbox[3]
        
        # Якщо виділено достатню область, зберігаємо шаблон
        if w > 10 and h > 10 and frame_gray is not None:
            roi_selected = True
            # Вирізаємо шаблон з поточного сірого кадру
            template = frame_gray[bbox[1]:bbox[1]+h, bbox[0]:bbox[0]+w].copy()
            print("Template selected!")

def apply_filters(img_gray, mode):
    """Застосовує фільтри відповідно до режиму."""
    if mode == MODE_LOW_PASS:
        # Низькочастотна фільтрація (Розмиття Гауса)
        # Прибирає шум, залишає низькі частоти (плавні переходи)
        return cv2.GaussianBlur(img_gray, (15, 15), 0)
    
    elif mode == MODE_HIGH_PASS:
        # Високочастотна фільтрація (Лапласіан)
        # Виділяє перепади яскравості (контури)
        laplacian = cv2.Laplacian(img_gray, cv2.CV_64F)
        # Конвертуємо назад у uint8 для відображення
        return cv2.convertScaleAbs(laplacian)
    
    else:
        # Звичайний монохром
        return img_gray

def main():
    global frame_gray, current_mode, frame_count

    cap = cv2.VideoCapture(0)
    cap.set(3, 640)
    cap.set(4, 480)

    cv2.namedWindow("Lab 4 - Correlation Tracking")
    cv2.setMouseCallback("Lab 4 - Correlation Tracking", mouse_callback)

    print("Controls:")
    print("  Mouse Drag - Select object to track")
    print("  1 - Monochrome Mode")
    print("  2 - Low Pass Filter (Blur)")
    print("  3 - High Pass Filter (Edges)")
    print("  S - Save current frame")
    print("  Q - Quit")

    while True:
        ret, frame = cap.read()
        if not ret: break
        
        frame_count += 1

        # 1. Конвертація в монохром (основа для обробки)
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # 2. Застосування фільтрації (залежно від режиму)
        processed_frame = apply_filters(frame_gray, current_mode)

        # 3. Кореляційне супроводження (якщо є шаблон)
        if roi_selected and template is not None:
            # Важливо: Template Matching краще працює на тому ж типі зображення,
            # на якому був створений шаблон. Але для лаби ми застосовуємо
            # пошук шаблону на фільтрованому зображенні.
            
            # Якщо ми в режимі фільтрів, треба або фільтрувати шаблон теж,
            # або шукати на чистому сірому. 
            # За завданням: "супроводження... при фільтрації". 
            # Тому шукаємо на processed_frame.
            
            # Захист: шаблон не може бути більшим за кадр
            if template.shape[0] <= processed_frame.shape[0] and \
               template.shape[1] <= processed_frame.shape[1]:
                
                # Основна функція кореляції
                res = cv2.matchTemplate(processed_frame, template, cv2.TM_CCOEFF_NORMED)
                
                # Знаходимо точку найкращого збігу (максимум кореляції)
                min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
                
                # max_loc - це лівий верхній кут знайдення
                top_left = max_loc
                h, w = template.shape
                bottom_right = (top_left[0] + w, top_left[1] + h)

                # Відображаємо прямокутник на кольоровому кадрі
                cv2.rectangle(frame, top_left, bottom_right, (0, 255, 0), 2)
                
                # Вивід координат
                center_x = top_left[0] + w // 2
                center_y = top_left[1] + h // 2
                coords_text = f"X: {center_x}, Y: {center_y}"
                cv2.putText(frame, coords_text, (top_left[0], top_left[1] - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                
                # Малюємо на обробленому кадрі теж (для наочності режимів)
                cv2.rectangle(processed_frame, top_left, bottom_right, 255, 2)

        # Якщо користувач зараз виділяє мишею
        if drawing:
             cv2.rectangle(frame, (bbox[0], bbox[1]), 
                           (bbox[0]+bbox[2], bbox[1]+bbox[3]), (255, 0, 0), 2)

        # Інфо на екрані
        info_text = f"Mode: {mode_names[current_mode]}"
        cv2.putText(frame, info_text, (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Відображаємо вікна
        cv2.imshow("Lab 4 - Correlation Tracking", frame)
        cv2.imshow("Processed View (Mono/Filter)", processed_frame)

        # Обробка клавіш
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('1'):
            current_mode = MODE_MONO
        elif key == ord('2'):
            current_mode = MODE_LOW_PASS
        elif key == ord('3'):
            current_mode = MODE_HIGH_PASS
        elif key == ord('s'):
            # Збереження кадру
            filename = f"{OUTPUT_FOLDER}/frame_{frame_count}.jpg"
            cv2.imwrite(filename, frame) # Зберігаємо оригінал з розміткою
            print(f"Frame saved: {filename}")

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Висновок

У ході виконання лабораторної роботи було досліджено та програмно реалізовано методи кореляційного супроводження об'єктів у відеопотоці.

Основні результати роботи:

Реалізація кореляційного трекінгу:

Розроблено програму, яка дозволяє користувачеві виділити довільний об'єкт на зображенні (ROI) та автоматично відстежувати його переміщення в наступних кадрах.

Використано метод Template Matching (cv2.matchTemplate), який шукає область у поточному кадрі, що має найвищий коефіцієнт кореляції з еталонним шаблоном. Це дозволяє супроводжувати об'єкти складної форми, які важко описати простими геометричними примітивами.

Робота з фільтрами:

Досліджено вплив частотної фільтрації на якість трекінгу:

Низькочастотна фільтрація (Low Pass / Blur): Зменшує рівень шуму, що корисно при поганому освітленні, але може знизити точність позиціонування через розмиття меж об'єкта.

Високочастотна фільтрація (High Pass / Edge Detection): Виділяє контури (перепади яскравості). Цей режим виявився ефективним для супроводження текстурних об'єктів та об'єктів з чіткими межами, оскільки він менш чутливий до змін загального рівня освітленості.

Взаємодія та збереження даних:

Реалізовано відображення поточних координат центру об'єкта в реальному часі.

Забезпечено можливість запису окремих кадрів з результатами роботи алгоритму у графічні файли для подальшого аналізу.

Підсумок:
Кореляційні методи є ефективним інструментом для задач комп'ютерного зору, де об'єкт не змінює суттєво свій масштаб або орієнтацію. Однак, було помічено, що класичний matchTemplate чутливий до поворотів об'єкта та зміни його розміру, що є обмеженням даного методу порівняно зі складнішими алгоритмами (наприклад, KCF або CSRT).